## Compute Metrics for one folder pair

In [ ]:
import logging
logging.getLogger().setLevel(logging.ERROR)
import os 
import glob
from PIL import Image
from typing import List, Optional, Union
import numpy as np
import torch
from metrics.metrics_old import Metrics, GeometryMetrics
from metrics.helpers import preprocess_image, preprocess_image_rgba, process_folder_new
import json
import pandas as pd

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
original_folder = "example_data/examples_08082025/Car_Sample_1/original"
generated_folder = "example_data/examples_08082025/Car_Sample_1/generated_side"

### Compute Semantic/Quality Metrics
- For evaluations with very small sample size (n<20), set *compute_distribution_metrics = False* as FID & KID require a minimum of 20 samples to be computed

In [ ]:
semantic_metrics = process_folder_new(
    original_folder=original_folder,
    generated_folder=generated_folder,
    preprocess_func=preprocess_image,  
    metric_class=Metrics,
    device="cuda",  
    compute_distribution_metrics=False  
)
print("Semantic Metrics:")
for k, v in semantic_metrics.items():
    print(f"{k}: {v}")

### Compute Geometric Evaluation Metrics

In [ ]:
# For geometry metrics:
geometry_metrics = process_folder_new(
    original_folder=original_folder,
    generated_folder=generated_folder,
    preprocess_func=preprocess_image_rgba,  
    metric_class=GeometryMetrics,
    num_points=100
)
print("Geometry Metrics:")
for k, v in geometry_metrics.items():
    print(f"{k}: {v}")

## Compute Metrics for multiple combinations

In [ ]:
from metrics.metrics_eval_old import process_metrics_by_viewpoint, tensor_to_serializable, json_file_to_combined_table

### Compute Metrics for a subfolder

In [ ]:
gt_folder = "example_data/Meshfleet_Eval/GT_test"
gen_folder = "example_data/Meshfleet_Eval/Results_Test"

metrics_result = process_metrics_by_viewpoint(gt_folder, gen_folder, device="cuda")
json_output = json.dumps(tensor_to_serializable(metrics_result), indent=4)

# Save the results in a JSON file in the root folder.
output_file = os.path.join(os.path.dirname(gt_folder), "metrics_results_001_test.json")
with open(output_file, 'w') as f:
    f.write(json_output)

print(f"Metrics saved to {output_file}")

### Print metrics from json into a table

In [ ]:
json_filepath = "example_data/Meshfleet_Eval/metrics_results_diagonal1.json"
combined_df = json_file_to_combined_table(json_filepath)

print("Combined Metrics Table:")
print(combined_df.to_string())
